# Worksheet 16

Name:  shengwen ouyang
UID: U01208928

### Topics

- Support Vector Machines (Non-linear case)

## Support Vector Machines

Follow along in class to implement the perceptron algorithm and create an animation of the algorithm.

a) As we saw in class, the form
$$w^T x + b = 0$$
while simple, does not expose the inner product `<x_i, x_j>` which we know `w` depends on, having done the math. This is critical to applying the "kernel trick" which allows for learning non-linear decision boundaries. Let's modify the above algorithm to use the form
$$\sum_i \alpha_i <x_i, x> + b = 0$$

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

epochs = 100
learning_rate = .05
expanding_rate = .99
retracting_rate = 1.1

X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

alpha_i = np.zeros((len(X),))
b = 0

def snap(x, alpha_i, b, error, X, Y, gamma):
    # create a mesh to plot in
    h = .01  # step size in the mesh
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    meshData = np.c_[xx.ravel(), yy.ravel()]
    cs = np.array([x for x in 'gb'])
    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)

    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='y',fill=False))
   
    Z = predict_many(alpha_i, b, meshData, X, Y, gamma)
    Z = np.array([0 if z <=0 else 1 for z in Z]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=.5, cmap=plt.cm.Paired)
    fig.savefig(TEMPFILE)
    plt.close()
    return im.fromarray(np.asarray(im.open(TEMPFILE)))

def predict(alpha_i, b, x, X, Y, gamma):
    return sum(alpha_i[i] * Y[i] * rbf_kernel(X[i], x, gamma) for i in range(len(X))) + b


def predict_many(alpha_i, b, Z, X, Y, gamma):
    # This will use the RBF kernel for predictions.
    return np.array([predict(alpha_i, b, z, X, Y, gamma) for z in Z])
gamma = 0.5
images = []
for epoch in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    error = False
    x, y = X[i], Y[i]
    
    # Calculate the prediction for the current point
    prediction = predict(alpha_i, b, x, X, Y, gamma)
    
    # Check if the prediction is correct
    if y * prediction <= 0:  # Misclassified point
        error = True
        # Update rule for the alpha coefficients and bias term
        alpha_i[i] += learning_rate * y
        b += learning_rate * y
    else:
        # If no error, let's try to marginally adjust the alphas downwards
        # to ensure we're not overfitting to the training data
        alpha_i[i] = max(0, alpha_i[i] * expanding_rate)
        b *= expanding_rate
    
    # Snap a picture of the current state
    images.append(snap(x, alpha_i, b, error, X, Y, gamma))


images[0].save(
    'svm_dual.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)

Write a configurable kernel function to apply in lieu of the dot product. Try it out on a dataset that is not linearly separable.

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

def rbf_kernel(x_i, x_j, gamma):
    return np.exp(-gamma * np.linalg.norm(x_i - x_j) ** 2)

# Your dataset creation and initial variables remain the same
def predict(alpha_i, b, x, X, Y, gamma):
    return sum(alpha_i[i] * Y[i] * rbf_kernel(X[i], x, gamma) for i in range(len(X))) + b


def predict_many(alpha_i, b, Z, X, Y, gamma):
    # This will use the RBF kernel for predictions.
    return np.array([predict(alpha_i, b, z, X, Y, gamma) for z in Z])




b) Assume we fit an SVM using a polynomial Kernel function and it seems to overfit the data. How would you adjust the tuning parameter `n` of the kernel function?

Adjusting the degree n of the polynomial kernel in an SVM is a powerful method to control model complexity and combat overfitting. It's essential to approach this adjustment methodically, using techniques like cross-validation to find the optimal balance between underfitting and overfitting. Remember, the goal is to achieve a model that generalizes well to unseen data, not one that performs exceptionally on the training data but poorly on new examples.

c) Assume we fit an SVM using a RBF Kernel function and it seems to underfit the data. How would you adjust the tuning parameter `sigma` of the kernel function?

Adjusting the σ parameter of the RBF kernel is a key method for addressing underfitting in SVM models. A larger σ increases the model's capacity to capture more complex patterns by allowing points to influence each other over greater distances. However, it's important to balance this adjustment with considerations of the regularization parameter and feature scaling to ensure optimal model performance. Always use systematic tuning and validation techniques like cross-validation to determine the best parameters for your model.

d) Tune the parameter of a specific Kernel function, to fit an SVM (using your code above) to the following dataset:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

data = np.loadtxt("spiral.data")
X, y = data[:, :2], data[:, 2]

plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title('Spiral Data')
plt.show()

clf = SVC(kernel='rbf')

gamma_range = [0.1, 0.5, 1, 2, 5, 10]


best_score = 0
best_gamma = 0

for gamma in gamma_range:
    clf.set_params(gamma=gamma)
    clf.fit(X, y)
    
    score = clf.score(X, y)
    
    if score > best_score:
        best_score = score
        best_gamma = gamma

print(f'Best Gamma: {best_gamma}')
